In [ ]:
# default_exp signUp

# SignUp

> Interface to sign up user

In [ ]:
#hide
from nbdev.showdoc import *
import src.setEnv 

In [ ]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from beartype import beartype
from uuid import uuid4
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from time import time

In [ ]:
#check whether hash password is working here
examplePassword1 = '123'
exampleHashedPassword1 = hash_password(examplePassword1)
exampleHashedPassword1

'f6055b9acd229f575c12a17eb31dd9e22cfbd5f1ffa60c9f1190b9d44549697d:162e28162b6a4b9e8b27b98f7a771801'

## User Input Class

In [ ]:
#export 
@dataclass_json
@dataclass
class UserInput:
    username: str
    password: str
    
    @property
    def passwordHash(self):
        return hash_password(self.password)
    
    def saveTable(self):
        passwordHashed = self.passwordHash
        userTable = UserTable(
                userId = str(uuid4()),
                username = self.username,
                date = time(),
                passwordHash = passwordHashed
            )
        userTable.save()
        

In [ ]:
example = UserInput.from_dict({'username': 'DelG', 'password': '123'})
example

UserInput(username='DelG', password='123')

## Main Lambda Handler

In [ ]:
#export 
class H:
    class ParseInputError(Exception): pass
    class SavingError(Exception): pass
    @classmethod
    @beartype
    def parseInput(cls, event:dict)->UserInput:
        try:
            user = Event.parseDataClass(UserInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def saveUserMethod(cls, user:UserInput)->bool:
        try:
            user.saveTable()
            return True
        except Exception as e:
            raise cls.SavingError(e)
    
        

In [ ]:
#export
def signUp(event, *args):
    try:
        user = H.parseInput(event)
        print(user.username)
        H.saveUserMethod(user)
        return Response.returnSuccess()
    except H.SavingError as e:
        return Response.returnError(f'failed saving user {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')
    

In [ ]:
input_ = {'username': 'Delgg', 'password': '1276'}
event = Event.getInput(body = input_)
signUp(event)

Delgg


{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
epoch = time()
epoch

1658677547.060287